# Initialization steps

In [1]:
import pandas as pd
from drg_connect import Snowflake
import qgrid 
from datetime import timedelta, datetime
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW',database='SANDBOX_ANALYTICS',schema = 'SANDBOX')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# ICD reference tables

In [3]:
snow.select("select * from RWD_DB.RWD.ICD_GROUPER limit 3")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,1,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A00,No map,No map,Cholera,Cholera,2017-11-02,2017-11-02
1,2,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A000,0010,Cholera due to vibrio cholerae,Cholera due to Vibrio cholerae 01 biovar cholerae,Cholera due to Vibrio cholerae 01 biovar cholerae,2017-11-02,2017-11-02
2,3,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A001,0011,Cholera due to vibrio cholerae el tor,Cholera due to Vibrio cholerae 01 biovar eltor,Cholera due to Vibrio cholerae 01 biovar eltor,2017-11-02,2017-11-02


In [4]:
%%read_sql

select
    *
from
    RWD_DB.RWD.ICD_GROUPER
where
    level_4_short_description_icd10 ilike '%chronic%'
    and level_4_short_description_icd10 ilike '%allograft%'

Query started at 07:11:32 PM India Standard Time; Query executed in 0.09 m

,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts


In [6]:
%%read_sql df

select
    *
from
    RWD_DB.RWD.ICD_GROUPER
where
    level_4_short_description_icd10 ilike '%kidney%'
    and level_4_short_description_icd10 ilike '%transplant%'

Query started at 07:12:54 PM India Standard Time; Query executed in 0.09 m

,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,82659,S00 - T889XXS,Injury poisoning and certain other consequence...,T80-T88,Complications of surgical and medical care not...,T86-T869,Complications of transplanted organs and tissue,T861,No map,No map,Complications of kidney transplant,Complications of kidney transplant,2017-11-02,2017-11-02
1,82660,S00 - T889XXS,Injury poisoning and certain other consequence...,T80-T88,Complications of surgical and medical care not...,T86-T869,Complications of transplanted organs and tissue,T8610,99681,Complications of transplanted kidney,Unspecified complication of kidney transplant,Unspecified complication of kidney transplant,2017-11-02,2017-11-02
2,82661,S00 - T889XXS,Injury poisoning and certain other consequence...,T80-T88,Complications of surgical and medical care not...,T86-T869,Complications of transplanted organs and tissue,T8611,99681,Complications of transplanted kidney,Kidney transplant rejection,Kidney transplant rejection,2017-11-02,2017-11-02
3,82662,S00 - T889XXS,Injury poisoning and certain other consequence...,T80-T88,Complications of surgical and medical care not...,T86-T869,Complications of transplanted organs and tissue,T8612,99681,Complications of transplanted kidney,Kidney transplant failure,Kidney transplant failure,2017-11-02,2017-11-02
4,82663,S00 - T889XXS,Injury poisoning and certain other consequence...,T80-T88,Complications of surgical and medical care not...,T86-T869,Complications of transplanted organs and tissue,T8613,No map,No map,Kidney transplant infection,Kidney transplant infection,2017-11-02,2017-11-02
5,82664,S00 - T889XXS,Injury poisoning and certain other consequence...,T80-T88,Complications of surgical and medical care not...,T86-T869,Complications of transplanted organs and tissue,T8619,No map,No map,Other complication of kidney transplant,Other complication of kidney transplant,2017-11-02,2017-11-02
6,93144,Z00 - Z9989,Factors influencing health status and contact ...,Z40-Z53,Encounters for other specific health care,Z48-Z488,Encounter for other postprocedural aftercare,Z4822,No map,No map,Encounter for aftercare following kidney trans...,Encounter for aftercare following kidney trans...,2017-11-02,2017-11-02
7,93989,Z00 - Z9989,Factors influencing health status and contact ...,Z77-Z99,Persons with potential health hazards related ...,Z94-Z949,Transplanted organ and tissue status,Z940,V420,Kidney replaced by transplant,Kidney transplant status,Kidney transplant status,2017-11-02,2017-11-02


In [8]:
# Export to check, and then will inactivate codes that are not relevant and re-upload as final reference table
df.to_excel("out/cai_kidney.xlsx", index=False)

In [9]:
%%read_sql df

select
    *
from
    RWD_DB.RWD.PELICAN_E_SNOMED_CONCEPT
limit 3

Query started at 07:30:01 PM India Standard Time; Query executed in 0.10 m

,concept_id,concept_status,fully_specified_name,ctvc_id,snomed_id,is_primitive
0,110882004,0,"Urethra and urethral meatus, male to female se...",XU2l9,T-DD202,1
1,110942005,0,Rectal cytologic material (specimen),XU2mG,T-EA552,0
2,111002006,0,Proteinaceous lymphadenopathy (disorder),XU2nB,DC-72132,1


In [10]:
%%read_sql df

select
    *
from
    RWD_DB.RWD.PELICAN_E_SNOMED_CONCEPT
where fully_specified_name ilike '%kidney%'
    and fully_specified_name ilike '%transplant%'

Query started at 07:32:42 PM India Standard Time; Query executed in 0.09 m

,concept_id,concept_status,fully_specified_name,ctvc_id,snomed_id,is_primitive
0,438804005,0,Tissue specimen from transplanted kidney (spec...,XUhsf,G-8442,0
1,428645009,0,Examination of recipient after kidney transpla...,XUfyB,P2-015A8,1
2,428575007,0,Hypertension secondary to kidney transplant (d...,XUfx3,D3-02212,0
3,440449002,0,Magnetic resonance angiography of transplanted...,XUiJC,P5-09087,1
4,316054003,10,[V]Kidney transplanted (situation),ZV420,R-21032,1
5,440512006,0,Biopsy of transplanted kidney using ultrasound...,XUiKD,P5-B7103,0
6,105462002,0,Donor of kidney for transplant (person),XU1TI,S-15012,1
7,70948008,0,Structure of transplanted kidney (body structure),XUJS4,T-7100A,1
8,284348003,0,Excision of rejected transplanted kidney (proc...,Xa2h5,P1-71400,1
9,175899003,0,Autotransplantation of kidney (procedure),7B000,P1-71D04,0


In [11]:
# Export to check, and then will inactivate codes that are not relevant and re-upload as final reference table
df.to_excel("out/kt_snomed.xlsx", index=False)

In [13]:
%%read_sql df

select
    *
from
    RWD_DB.RWD.PELICAN_E_SNOMED_CONCEPT
where fully_specified_name ilike '%chronic%'
    and fully_specified_name ilike '%allograft%'

Query started at 07:41:03 PM India Standard Time; Query executed in 0.05 m

,concept_id,concept_status,fully_specified_name,ctvc_id,snomed_id,is_primitive
0,86825002,0,Chronic arterial allograft rejection (finding),XUMdp,F-CC460,1


In [12]:
%%read_sql df

select
    *
from
    RWD_DB.RWD_REFERENCE_LIBRARY.HCPCS_BASE_LIBRARY
where 
     (full_description ilike '%chronic%'
      and full_description ilike '%allograft%')  

Query started at 07:39:30 PM India Standard Time; Query executed in 0.08 m

,hcpcs_code,short_description,long_description,full_description,status,source_file_name


In [14]:
%%read_sql df

select
    *
from
    RWD_DB.RWD_REFERENCE_LIBRARY.CPT_BASE_LIBRARY limit 3
--where 
     --(full_description ilike '%chronic%'
      --and full_description ilike '%allograft%')  

Query started at 07:44:44 PM India Standard Time; Query executed in 0.08 m

,cpt_code,short_description,long_description,full_description,nf_total_rvu,fac_total_rvu,status,resequenced,seq_order,source_file_name
0,0001F,HEART FAILURE COMPOSITE,HRT FAILURE ASSESSED,Heart failure assessed (includes assessment of...,0.00,0.00,None,None,9312,OPTUM_CPC_2016_01_with_October_2015_RVUs.zip
1,0001M,INFECTIOUS DIS HCV 6 ASSAYS,INFECTIOUS DIS HCV 6 ASSAYS SERUM LIVER,"Infectious disease, HCV, six biochemical assay...",0.00,0.00,None,None,10122,OPTUM_CPC_2016_01_with_October_2015_RVUs.zip
2,0002M,LIVER DIS 10 ASSAYS W/ASH,LIVER DIS 10 ASSAYS SERUM ALGORITHM W/ASH,"Liver disease, ten biochemical assays (ALT, A2...",0.00,0.00,None,None,10123,OPTUM_CPC_2016_01_with_October_2015_RVUs.zip


In [15]:
%%read_sql

select
    *
from
    RWD_DB.RWD.ICD_GROUPER
where
    level_4_short_description_icd10 ilike '%Unspecified%'
    and level_4_short_description_icd10 ilike '%nephritic%'
    and level_4_short_description_icd10 ilike '%syndrome%'

Query started at 08:11:06 PM India Standard Time; Query executed in 0.07 m

,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,23483,N00 - N9989,Diseases of the genitourinary system,N00-N08,Glomerular diseases,N05-N059,Unspecified nephritic syndrome,N05,No map,No map,Unspecified nephritic syndrome,Unspecified nephritic syndrome,2017-11-02,2017-11-02
1,23490,N00 - N9989,Diseases of the genitourinary system,N00-N08,Glomerular diseases,N05-N059,Unspecified nephritic syndrome,N056,No map,No map,Unspecified nephritic syndrome with dense depo...,Unspecified nephritic syndrome with dense depo...,2017-11-02,2017-11-02
2,23493,N00 - N9989,Diseases of the genitourinary system,N00-N08,Glomerular diseases,N05-N059,Unspecified nephritic syndrome,N059,5830,Nephritis and nephropathy not specified as acu...,Unsp nephritic syndrome with unspecified morph...,Unspecified nephritic syndrome with unspecifie...,2017-11-02,2017-11-02


In [19]:
%%read_sql
select
    *
from
    RWD_DB.RWD.PELICAN_DIAGNOSIS_ICD10

    where

    icd10 in ('N059', 'N05', 'N058', 'N056')

Query started at 08:19:37 PM India Standard Time; Query executed in 0.06 m

,diagnosis_id,icd10,etl_icd10


In [25]:
%%read_sql
create or replace table st_cai1 as
select
    patient_id,
    left(year_of_service, 4) as year,
    claim_number,
    diagnosis,
    diagnosis_sequence
    
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS
where
    diagnosis in ('N059')

Query started at 08:39:51 PM India Standard Time; Query executed in 0.16 m

,status
0,Table ST_CAI1 successfully created.


In [26]:
%%read_sql
select count (distinct patient_id)
from
    st_cai1
    
where 
    left(patient_id, 5) != 'XXX -'

Query started at 08:40:03 PM India Standard Time; Query executed in 0.12 m

,COUNT (DISTINCT PATIENT_ID)
0,59862


In [24]:
%%read_sql
select diagnosis,
       count (distinct patient_id)
from
    st_cai1
    
where 
    left(patient_id, 5) != 'XXX -'
group by 1
order by 1

Query started at 08:24:00 PM India Standard Time; Query executed in 0.08 m

,diagnosis,COUNT (DISTINCT PATIENT_ID)
0,N05,13
1,N056,311
2,N058,23451
3,N059,59862


# CPT codes
using CPT codes: 90951-90970

In [29]:
%%read_sql
create or replace table st_cai_cpt1 as
select
    patient_id,
    left(year_of_service, 4) as year,
    claim_number,
    procedure
    
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROCEDURE
where
    procedure in ('90951', '90952', '90953', '90954', '90955', '90956', '90957', '90958', '90959', '90960', 
                  '90961', '90962', '90963', '90964', '90965', '90966', '90967', '90968', '90969', '90970')
    and patient_id in (select distinct patient_id from st_cai1)

Query started at 08:41:45 PM India Standard Time; Query executed in 0.32 m

,status
0,Table ST_CAI_CPT1 successfully created.


In [30]:
%%read_sql
select count (distinct patient_id)
from
    st_cai_cpt1
    
where 
    left(patient_id, 5) != 'XXX -'

Query started at 08:42:27 PM India Standard Time; Query executed in 0.06 m

,COUNT (DISTINCT PATIENT_ID)
0,5103
